In [1]:
!pip install transformers # Installing the transformers library

     |████████████████████████████████| 1.8MB 16.3MB/s 
     |████████████████████████████████| 890kB 57.3MB/s 
     |████████████████████████████████| 2.9MB 57.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0ce91f8647585aaee37cc8c04a455ce1827efae525bc09bf82bb023f35c0dcf4
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import transformers # transformers library
import torch # PyTorch, we are using PyTorch as our library

In [3]:
# We are going to load in GPT-2 using the transformers library
gpt_tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2-large')
# Loading in model now...
gpt_model = transformers.GPT2LMHeadModel.from_pretrained('gpt2-large')
# Takes a while to run...

In [4]:
## Making a function that will generate text for us ##
def gen_text(prompt_text, tokenizer, model, n_seqs=1, max_length=25):
  # n_seqs is the number of sentences to generate
  # max_length is the maximum length of the sentence
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  # We are encoding the text using the gpt tokenizer. The return tensors are of type "pt"
  # since we are using PyTorch, not tensorflow
  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=max_length+len(encoded_prompt), # The model has to generate something, 
      # so we add the length of the original sequence to max_length
      temperature=1.0,
      top_k=0,
      top_p=0.9,
      repetition_penalty=1.2, # To ensure that we dont get repeated phrases
      do_sample=True,
      num_return_sequences=n_seqs
  ) # We feed the encoded input into the model.
  ## Getting the output ##
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_() # the _ indicates that the operation will be done in-place
  generated_sequences = []
  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence)
    total_sequence = (
        prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True, )) :]
    )
    generated_sequences.append(total_sequence)
  return generated_sequences

In [5]:
# Lots of syntax errors, but now we can test our model
## One important note: in our function, on line 5, make sure that
# return_tensor is return_tensors, otherwise you will get an error like
# this:
#####
# Another important note: on line 27 of the function, instead of
# clear_up_tokenization_spaces, write clean_up_tokenization_spaces
####
gen_text("Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry",gpt_tokenizer,gpt_model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry and targeting the enemy forces.\n\n']

In [6]:
# Sequence length was too small, lets increase it
gen_text("Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry",
         gpt_tokenizer,
         gpt_model,
         max_length=100)
# Will take some time......

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. Catching sight of two more giant battleaxes that had been flung into the darkness, Rohan fell back from his rapid advance to reload his double-barreled shotguns. The orc deadclawed them easily, but the other armored great berserker leapt at the beleaguered Merry and Pippin. His axe missed its mark, but the flailing arms seemed to surprise him; he']

In [7]:
# We can demostrate n_seqs here
gen_text("Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry",
         gpt_tokenizer,
         gpt_model,
         max_length=40,
         n_seqs=3) # Will take even longer....

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.\n\n"Don\'t let your foolish gods tell you that being a part of the Night\'s Watch is easy',
 'Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. They gave them two clear choices: fight or die!\n\nThe orc captains raised their axes in surrendering',
 'Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. As their final volley flew they were severely wounded by a small group of Riders - two mounted archers against two']

In [ ]:
# There are now 3 different outputs
# thats it